### 1. Extract covid vaccination data of states in India from public databases. Provide a descriptive summary of the top 10 districts throughout India with highest vaccination count for the 1st week of July.

### 2. Choose top 3 districts from above analysis and create labelled time series plots for these districts.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import warnings
import plotly.express as px
from datetime import date, timedelta
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
df=pd.read_csv("../input/covid-vaccination-india-district-wise-data/cowin_vaccine_data_districtwise.csv")

In [ ]:
df.head()

In [ ]:
df.iloc[0]=df.iloc[0].replace(np.nan,0)

In [ ]:
list1=[]
for i,j in zip(df.columns,df.iloc[0]):
    if j==0 or 0.0:
        list1.append(str(i))
    else:
        list1.append(str(i)+"_"+str(j))

In [ ]:
df.columns=list1

In [ ]:
df.drop(0,inplace=True,axis=0)

In [ ]:
metadata=df.iloc[:,1:6]

In [ ]:
firts_week_july_data=df.iloc[:,1666:1736]

In [ ]:
final_data_july=pd.concat([metadata,firts_week_july_data],axis=1)
final_data_july.reset_index(inplace=True,drop=True)
t=final_data_july[final_data_july.isnull().any(1)]["01/07/2021.3_First Dose Administered"].astype("int").values[0]+final_data_july[final_data_july.isnull().any(1)]["01/07/2021.4_Second Dose Administered"].astype("int").values[0]
final_data_july.iloc[6]["01/07/2021_Total Doses Administered"]=final_data_july[final_data_july.isnull().any(1)]["01/07/2021_Total Doses Administered"].replace(np.nan,t)
t1=final_data_july.iloc[:,5:].astype("int")
final_df=pd.concat([final_data_july.iloc[:,:5],t1],axis=1)

In [ ]:
final_df[final_df.isnull().any(1)]

In [ ]:
t=final_df.iloc[0]["01/07/2021.3_First Dose Administered"]+final_df.iloc[0]["01/07/2021.4_Second Dose Administered"]
final_df.iloc[0,5]=t

In [ ]:
final_dataset1=final_df.filter(regex="_Total Doses Administered")
final_dataset1["Dsitrict"]=final_df["District"]
final_dataset1["Total_vaccination_by_distric"]=final_dataset1.iloc[:,:7].sum(axis=1)

In [ ]:
l1=["District"]
l2=list(final_df.filter(regex="_Total Doses Administered").columns.values)

In [ ]:
l1.extend(l2)

In [ ]:
t3=final_df.groupby("District").sum()
dates=[]
for i in l2:
    dates.append(i.split("_")[0])

In [ ]:
def discrptive_summary_per_district_for_first_week_july(t3,district,dates,final_df):
    l1=list(final_df.filter(regex="_Total").columns.values)
    l2=list(final_df.filter(regex="_First").columns.values)
    l3=list(final_df.filter(regex="_Second").columns.values)
    l5=list(final_df.filter(regex="_Male").columns.values)
    l6=list(final_df.filter(regex="_Female").columns.values)
    l7=list(final_df.filter(regex="_Transgender").columns.values)
    l8=list(final_df.filter(regex="_Covaxin ").columns.values)
    l9=list(final_df.filter(regex="_CoviShield ").columns.values)
    example_df1=pd.DataFrame()
    example_df1["Dates"]=dates
    example_df1["Total Doses Administered"]=t3[t3.index==district][l1].values[0]
    example_df1["First Dose Administered"]=t3[t3.index==district][l2].values[0]
    example_df1["Second Dose Administered"]=t3[t3.index==district][l3].values[0]
    example_df1["Male(Doses Administered)"]=t3[t3.index==district][l5].values[0]
    example_df1["Female(Doses Administered)"]=t3[t3.index==district][l6].values[0]
    example_df1["Transgender(Doses Administered)"]=t3[t3.index==district][l7].values[0]
    example_df1["Covaxin (Doses Administered)"]=t3[t3.index==district][l8].values[0]
    example_df1["CoviShield (Doses Administered)"]=t3[t3.index==district][l9].values[0]
    t=example_df1.describe()
    return t,example_df1

In [ ]:
district_df=pd.DataFrame()

In [ ]:
district_df["Total Doses Administered"]=t3.filter(regex="_Total").sum(axis=1)
district_df["First Doses Administered"]=t3.filter(regex="_First").sum(axis=1)
district_df["Second Dose Administered"]=t3.filter(regex="_Second").sum(axis=1)
district_df["Male(Doses Administered)"]=t3.filter(regex="_Male").sum(axis=1)
district_df["Female(Doses Administered)"]=t3.filter(regex="_Female").sum(axis=1)
district_df["Female(Doses Administered)"]=t3.filter(regex="_Female").sum(axis=1)
district_df["Transgender(Doses Administered)"]=t3.filter(regex="_Transgender").sum(axis=1)
district_df["Covaxin (Doses Administered)"]=t3.filter(regex="_Covaxin ").sum(axis=1)
district_df["CoviShield (Doses Administered)"]=t3.filter(regex="_CoviShield").sum(axis=1)

In [ ]:
top_10_districts=district_df.sort_values(by="Total Doses Administered",ascending=False)

In [ ]:
top_10_districts=top_10_districts.head(10)

## Descrptive summary for Top 10 Districts

In [ ]:
top_10_districts.describe()

In [ ]:
male_sum=top_10_districts["Male(Doses Administered)"].values.sum()
female_sum=top_10_districts["Female(Doses Administered)"].values.sum()
trans_sum=top_10_districts["Transgender(Doses Administered)"].values.sum()

In [ ]:
    px.pie(names=["Male Vaccinated","Female Vaccinated","Trans Gender"],values=[male_sum,female_sum,trans_sum],title="Male Female and transgender Vaccinated ratio for Covid19")

In [ ]:
covaxin=top_10_districts["Covaxin (Doses Administered)"].values.sum()
covi=top_10_districts["CoviShield (Doses Administered)"].values.sum()
px.pie(names=["Covaxin Vaccinated","CoviShield Vaccinated"],values=[covaxin,covi],title="Covaxin and CoviShield Vaccinated ratio for Covid19")

In [ ]:
first=top_10_districts["First Doses Administered"].values.sum()
second=top_10_districts["Second Dose Administered"].values.sum()
px.pie(names=["fist vaccine","second Vaccine"],values=[first,second],title="First and Second Vaccinated ratio for Covid19")

In [ ]:
fig=px.bar(top_10_districts,x=top_10_districts.index,y=top_10_districts["Total Doses Administered"],color=top_10_districts["Total Doses Administered"])
fig.update_layout(title_text="Most Doses Administrated")
fig.show()

In [ ]:
fig=make_subplots(rows=1,cols=2,subplot_titles=('First dose','Second dose'))
trace1=fig.add_trace(go.Bar(x=top_10_districts.index,y=top_10_districts["First Doses Administered"],marker_color=top_10_districts["First Doses Administered"],name="First_dose"),row=1,col=1)
trace2=fig.add_trace(go.Bar(x=top_10_districts.index,y=top_10_districts["Second Dose Administered"],marker_color=top_10_districts["Second Dose Administered"],name="Second_dose"),row=1,col=2)
fig.update_layout(xaxis_title="",yaxis_title="Y Axis Title",title_text="First v/s second dose")
fig['layout']['xaxis']['title']='District'
fig['layout']['xaxis2']['title']='District'
fig['layout']['yaxis']['title']='First Doses Administered'
fig['layout']['yaxis2']['title']='Second Dose Administered'

In [ ]:
fig.show()

In [ ]:
fig=make_subplots(rows=1,cols=2,subplot_titles=('Male','Female'))
trace1=fig.add_trace(go.Bar(x=top_10_districts.index,y=top_10_districts["Male(Doses Administered)"],marker_color=top_10_districts["Male(Doses Administered)"],name="Male"),row=1,col=1)
trace2=fig.add_trace(go.Bar(x=top_10_districts.index,y=top_10_districts["Female(Doses Administered)"],marker_color=top_10_districts["Female(Doses Administered)"],name="Female"),row=1,col=2)
fig.update_layout(xaxis_title="",yaxis_title="Y Axis Title",title_text="Male vs Female")
fig['layout']['xaxis']['title']='District'
fig['layout']['xaxis2']['title']='District'
fig['layout']['yaxis']['title']='Male(Doses Administered)'
fig['layout']['yaxis2']['title']='Female(Doses Administered)'

In [ ]:
fig.show()

In [ ]:
fig=make_subplots(rows=1,cols=2,subplot_titles=('Covaxin','CoviShield'))
trace1=fig.add_trace(go.Bar(x=top_10_districts.index,y=top_10_districts["Covaxin (Doses Administered)"],marker_color=top_10_districts["Covaxin (Doses Administered)"],name="Covaxin"),row=1,col=1)
trace2=fig.add_trace(go.Bar(x=top_10_districts.index,y=top_10_districts["CoviShield (Doses Administered)"],marker_color=top_10_districts["CoviShield (Doses Administered)"],name="CoviShield"),row=1,col=2)
fig.update_layout(xaxis_title="",yaxis_title="Y Axis Title",title_text="Covaxin v/s CoviShield")
fig['layout']['xaxis']['title']='District'
fig['layout']['xaxis2']['title']='District'
fig['layout']['yaxis']['title']='Covaxin (Doses Administered)'
fig['layout']['yaxis2']['title']='CoviShield (Doses Administered)'

In [ ]:
fig.show()

In [ ]:
fig=px.bar(top_10_districts,x=top_10_districts.index,y=top_10_districts["Transgender(Doses Administered)"],color=top_10_districts["Transgender(Doses Administered)"])
fig.update_layout(title_text="Transgender(Doses Administered)")
fig.show()

### TOP 3 DISTRICTS : Bengaluru Urban , Mumbai , Pune

In [ ]:
Bengakuru_data=discrptive_summary_per_district_for_first_week_july(t3,"Bengaluru Urban",dates,final_df)[1]
Mumbai_data=discrptive_summary_per_district_for_first_week_july(t3,"Mumbai",dates,final_df)[1]
Pune_data=discrptive_summary_per_district_for_first_week_july(t3,"Pune",dates,final_df)[1]

In [ ]:
Bengakuru_data

In [ ]:
Total_administered_dose=pd.DataFrame()
sdate=date(2021,7,1)
edate=date(2021,7,8)
Dates1=pd.date_range(sdate,edate-timedelta(days=1),freq='d')
Total_administered_dose["Dates"]=Dates1
Total_administered_dose["Total Doses Administered_Bengaluru"]=Bengakuru_data["Total Doses Administered"]
Total_administered_dose["Total Doses Administered_Mumbai"]=Mumbai_data["Total Doses Administered"]
Total_administered_dose["Total Doses Administered_Pune"]=Pune_data["Total Doses Administered"]

In [ ]:
Total_administered_dose

In [ ]:
fig=px.line(Total_administered_dose,x="Dates",y=Total_administered_dose.columns[1:],range_x=['2021-07-01', '2021-07-07'])
fig1=px.scatter(Total_administered_dose,x="Dates",y=Total_administered_dose.columns[1:],range_x=['2021-07-01', '2021-07-07'])
fig2=go.Figure(fig.data+fig1.data)
fig2.update_layout(
    title_text="Time series plot for 1st week of july for Top 3 districts"
)
fig2.show()

In [ ]:
df2=df[(df["District"]=="Bengaluru Urban") | (df["District"]=="Mumbai") | (df["District"]=="Pune")].iloc[:,5:]

In [ ]:
d=df2.iloc[:,1:].astype("int")

In [ ]:
d["District"]=df2["District"]

In [ ]:
df2=d.groupby("District").sum()

### For Total Doses adminstered in top 3 Districts

In [ ]:
Total_Doses_Administered=df2.filter(like="Total")

In [ ]:
Total_Doses_Administered

In [ ]:
Total_Doses_Administered.reset_index(inplace=True)

In [ ]:
Total_Doses_Administered=Total_Doses_Administered.T

In [ ]:
Total_Doses_Administered.reset_index(inplace=True)

In [ ]:
Total_Doses_Administered.columns=list(Total_Doses_Administered.iloc[0])

In [ ]:
Total_Doses_Administered.drop(0,axis=0,inplace=True)

In [ ]:
Total_Doses_Administered.reset_index(inplace=True)

In [ ]:
del Total_Doses_Administered['index']

In [ ]:
Total_Doses_Administered.rename(columns={"District":"Dates"},inplace=True)

In [ ]:
sdate=date(2021,1,16)
edate=date(2021,8,2)
Dates=pd.date_range(sdate,edate-timedelta(days=1),freq='d')

In [ ]:
Total_Doses_Administered["Dates"]=Dates

In [ ]:
fig1=px.line(Total_Doses_Administered,x=Total_Doses_Administered["Dates"],y=Total_Doses_Administered["Bengaluru Urban"],color_discrete_sequence=["red"])
fig2=px.line(Total_Doses_Administered,x=Total_Doses_Administered["Dates"],y=Total_Doses_Administered["Mumbai"],color_discrete_sequence=["green"])
fig3=px.line(Total_Doses_Administered,x=Total_Doses_Administered["Dates"],y=Total_Doses_Administered["Pune"],color_discrete_sequence=["yellow"])
fig3=go.Figure(fig1.data+fig2.data+fig3.data)
fig3.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(step="all")
        ])
    )
)

In [ ]:
def function(df2,Dates,Dates1,data1,data2):
    data1_df=df2.filter(like=data1)
    data2_df=df2.filter(like=data2)
    data1_df.reset_index(inplace=True)
    data2_df.reset_index(inplace=True)
    data1_df=data1_df.T
    data2_df=data2_df.T
    data1_df.reset_index(inplace=True)
    data2_df.reset_index(inplace=True)
    data1_df.columns=list(data1_df.iloc[0])
    data2_df.columns=list(data2_df.iloc[0])
    data1_df.drop(0,axis=0,inplace=True)
    data2_df.drop(0,axis=0,inplace=True)
    data1_df.reset_index(inplace=True)
    data2_df.reset_index(inplace=True)
    del data1_df['index']
    del data2_df['index']
    data1_df.rename(columns={"District":"Dates"},inplace=True)
    data2_df.rename(columns={"District":"Dates"},inplace=True)
    data1_df["Dates"]=Dates
    data2_df["Dates"]=Dates
    return data1_df,data2_df

In [ ]:
d1,d2=function(df2,Dates,Dates1,"Covaxin","CoviShield")

In [ ]:
def plot_subplots(d1,data1):
    fig1=px.line(d1,x=d1["Dates"],y=d1["Bengaluru Urban"],color_discrete_sequence=["red"])
    fig2=px.line(d1,x=d1["Dates"],y=d1["Mumbai"],color_discrete_sequence=["green"])
    fig3=px.line(d1,x=d1["Dates"],y=d1["Pune"],color_discrete_sequence=["yellow"])
    fig3=go.Figure(fig1.data+fig2.data+fig3.data)
    fig3.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(step="all")
            ])
        )
    )
    fig3.update_layout(
    title_text="Time series plot for 1st week of july for Top 3 districts for {}".format(data1)
    )
    fig3.show()

In [ ]:
plot_subplots(d1,"Covaxin")

In [ ]:
plot_subplots(d2,"CoviSheild")

In [ ]:
plot_subplots(d2,"Male")

In [ ]:
plot_subplots(d2,"Female")

In [ ]:
plot_subplots(d2,"First Doses")

In [ ]:
plot_subplots(d2,"Second Doses")

## 3. Create an algorithm in Python from scratch that can create a normal distribution from the data [0,0,0,0,0,0,0,0,0,0,10,10,10,10,10,10,10,10,10,10] while the total sum remains unchanged and plot the two curves.

In [ ]:
def normal_plot(x,l):
    mean=0
    std=0
    for i in x:
        mean+=i
    mean=mean/len(x)
    min1=min(x)
    max1=max(x)
    array=[min1 + x*(max1-min1)/(l-1) for x in range(0,l)]
    for i in x:
        s=i-mean
        s=s**2
        std+=s
    std=std/len(x)
    std=np.sqrt(std)
    list1=[]
    for i in array:
        exp_part=((i-mean)/std)**2
        exp_part=(1/2)*exp_part
        exp_part=np.exp(-exp_part)
        den=std*(np.sqrt(2*3.14))
        den=1/den
        final=den*exp_part
        list1.append(final)
    return list1

In [ ]:
x=[0,0,0,0,0,0,0,0,0,0,10,10,10,10,10,10,10,10,10,10]
l=100

In [ ]:
list1=normal_plot(x,l)

In [ ]:
fig=px.line(list1,color_discrete_sequence=["yellow"])
fig.update_xaxes(title="Data range")
fig.update_yaxes(title="Values")
fig.show()